# Collecting Raw Data

In [6]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import patsy
import matplotlib
import matplotlib.pyplot as plt
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
import random
import sys
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.style.use('fivethirtyeight')
from IPython.display import display
from bs4 import BeautifulSoup

In [7]:
# import user_ids

real_users1 = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/real_users1.csv')
real_users2 = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/real_users2.csv')
real_users3 = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/real_users3.csv')
real_users4 = pd.read_csv('/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/real_users4.csv')

In [8]:
# get list of unique user_ids

users = pd.DataFrame()
users = users.append(real_users1,ignore_index=True)
users = users.append(real_users2,ignore_index=True)
users = users.append(real_users3,ignore_index=True)
users = users.append(real_users4,ignore_index=True)
user_id_list = users['0'].unique()

In [9]:
# init DataFrames 

revs_df = pd.DataFrame()
users_df = pd.DataFrame()
hostels_df = pd.DataFrame()

# Scrape for Users and Revs Data

In [10]:
# Get details for user and rev DFs

for i,user in enumerate(user_id_list[50:]):
    
    rv_df = pd.DataFrame()
    us_df = pd.DataFrame()
    X = 'http://www.hostelworld.com/profile/' + str(user) + '/reviews'
    response = requests.get(X)
    HTML = response.text
    
    rv_df['text'] = Selector(text=HTML).xpath('//div[@id]/p/text()').extract()
    rv_df['user_id'] = [user]*len(rv_df['text'])
    rv_df['score'] = Selector(text=HTML).xpath("//div[@class='textrating']/text()").extract()
    rv_df['hostel'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/a/text()").extract()
    rv_df['location'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/span/text()").extract()
    rv_df['link'] = Selector(text=HTML).xpath("//div[@class='popupreviewlocation']/a/@href").extract()
    
    us_df['user_id'] = [user]
    us_df['about'] = [Selector(text=HTML).xpath("//li[@class='reviewerdetails']/text()").extract()[0]] 
    us_df['num_revs']=len(rv_df['text'])
    # us_df['avg_score'] = np.mean(df1['score']) 
  
    
    if i%10 == 0:
        sys.stdout.write('\r'+ str((float(i)/len(user_id_list))*100)+'%')
     
    users_df = users_df.append(us_df,ignore_index=True)
    revs_df = revs_df.append(rv_df,ignore_index=True)
    

98.6696230599%

In [ ]:

users_df.shape, revs_df.shape

In [ ]:
# save user and rev dfs

users_df.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/raw_users3.csv', encoding='utf8')
revs_df.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/raw_revs3.csv',encoding='utf8')

# Scrape for Hostels Data

In [ ]:
# Get data for hostels 

for i,url in enumerate(revs_df.link):
    hst_df = pd.DataFrame()
    X = str(url)
    response = requests.get(X)
    HTML = response.text
    
    free = Selector(text=HTML).xpath('//li[1]/div/p/text()').extract()
    general = Selector(text=HTML).xpath('//li[2]/div/p').extract()
    services = Selector(text=HTML).xpath('//li[3]/div/p').extract()
    food_drink = Selector(text=HTML).xpath('//li[4]/div/p').extract()
    entertainment = Selector(text=HTML).xpath('//li[5]/div/p').extract()
    policies = Selector(text=HTML).xpath("//div[@id='panel4']/p").extract()
    ratings = Selector(text=HTML).xpath("//div[@class='ratingbox']/text()").extract()
    description = Selector(text=HTML).xpath("//div[@class='section-propertytext']/div[@class='prop-intro']/div[@class='microdetailstext hiddentext prop-text bigtext ']/text()").extract()
    hst = revs_df.hostel.iloc[i]
    loc = revs_df.location.iloc[i]
    
    if i%10 == 0:
        sys.stdout.write('\r'+ str((float(i)/len(revs_df.link))*100)+'%')
        
    if len(ratings) != 0:
        if float(ratings[0]) != 0.0:

            hst_df['rating'] = [float(ratings[0])]
            hst_df['desc'] = [description]
            hst_df['FREE'] = [free]
            hst_df['GENERAL'] = [general] 
            hst_df['SERVICES'] = [services]
            hst_df['FOOD_DRINK'] = [food_drink]
            hst_df['ENTERTAINMENT'] = [entertainment]
            hst_df['POLICIES'] = [policies]
            hst_df['hostel']= hst
            hst_df['location']= loc
        
            hostels_df = hostels_df.append(hst_df,ignore_index=True)
            
        # else statment to del a rev from revs_df is the hostel no longer exists
        else:
            revs_df.drop(revs_df.index[i], inplace=True)
    # else statment to del a rev from revs_df is the hostel no longer exists
    else:
        revs_df.drop(revs_df.index[i], inplace=True)

0.420698359276%

In [ ]:

users_df.shape, revs_df.shape, hostels_df.shape

In [ ]:
# save hostels df

hostels_df.to_csv(path_or_buf='/Users/caitlinmowdy/Desktop/DSI-SF-2-caitlinmowdy/capstone-hostelworld/raw-data/raw_hostels3.csv',encoding='utf8')